In [1]:
%load_ext line_profiler

import csv,operator,sys,os
import numpy as np
import sklearn
import json

from functools import reduce

sys.path.append('../arch-forest/data/adult/')
sys.path.append('../arch-forest/data/')
sys.path.append('../arch-forest/code/')
import trainForest
import Tree

import DecisionSnippetFeatures

In [2]:
# grab adult training data in the format used by the random forests in forests/adult/text/

X = []
Y = []

f = open("../data/adult/adult.data")
for row in f:
    if (len(row) > 1):
        entries = row.replace("\n", "").replace(" ", "").split(",")

        x = trainForest.getFeatureVector(entries)

        if (entries[-1] == "<=50K"):
            y = 0
        else:
            y = 1

        Y.append(y)
        X.append(x)

X = np.array(X).astype(dtype=np.int32)
Y = np.array(Y)
f.close()

In [14]:
# load one of the readily available decision trees
f = open('../forests/adult/text/RF_5.json')
dt = json.load(f)
f.close()


In [15]:
# this creates a transformation of the dataset, assigning each example the vertex id of the leaf vertex 
# the example turns out at. So far, so boring
dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)
fts = dsf.fit_transform(X[:10, :])
print(fts)

[[39  6 23  5 45  5  5  5  6 41 37 53  5 12 20 36 28 15  5 15 35 13  8 20
   9]
 [34  8 50 24 52 38 56  6 59 41 50 42  8 32 47 23 20 13 58 12 12 28  8 12
   5]
 [39  5  8  5  6  5  5 16  5 12 34  6  5 18  6  5  8  5 15  5 34  6  8  5
   6]
 [42  8 36 20 25 30 49  6 36 13 47  6  5 32  5 21  6  8 36  5 12 18  8  6
   5]
 [26  8 21  8 42 40 27  6 23 36  9 41 32 36  5 27 20 23 30 12 12 12 36 15
   5]
 [42  8 21  8 42 40 12  6 23  9 41 41 39 36 37 23 20 13 20 21 12 12 43 15
   5]
 [34  5  8  6  6  5  5  5  5  5 34  6 32 18  5  5  5  5  6  5 34  6 46  5
  41]
 [54 40 50 21 21 30 56  9 51 13 48  6  8 45 47 21  9 37 51  8 12 22 27  6
  13]
 [28 43 20  5 45 25 21 25 12 31 35 50 40 22 29 37 44 52  9 31 50 14 37 21
  12]
 [42 27 51 24 52 38 56 20 58 41 50 44  9 32 52 23 38 13 60 12 15 28 16 13
   5]]


In [16]:
from sklearn.preprocessing import OneHotEncoder
fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)
print(fts_onehot)

  (0, 39)	1.0
  (0, 65)	1.0
  (0, 137)	1.0
  (0, 178)	1.0
  (0, 275)	1.0
  (0, 296)	1.0
  (0, 353)	1.0
  (0, 412)	1.0
  (0, 460)	1.0
  (0, 558)	1.0
  (0, 613)	1.0
  (0, 686)	1.0
  (0, 699)	1.0
  (0, 765)	1.0
  (0, 830)	1.0
  (0, 899)	1.0
  (0, 940)	1.0
  (0, 974)	1.0
  (0, 1023)	1.0
  (0, 1094)	1.0
  (0, 1171)	1.0
  (0, 1210)	1.0
  (0, 1266)	1.0
  (0, 1341)	1.0
  (0, 1385)	1.0
  :	:
  (9, 42)	1.0
  (9, 86)	1.0
  (9, 165)	1.0
  (9, 197)	1.0
  (9, 282)	1.0
  (9, 329)	1.0
  (9, 404)	1.0
  (9, 427)	1.0
  (9, 512)	1.0
  (9, 558)	1.0
  (9, 626)	1.0
  (9, 677)	1.0
  (9, 703)	1.0
  (9, 785)	1.0
  (9, 862)	1.0
  (9, 886)	1.0
  (9, 950)	1.0
  (9, 972)	1.0
  (9, 1078)	1.0
  (9, 1091)	1.0
  (9, 1151)	1.0
  (9, 1225)	1.0
  (9, 1274)	1.0
  (9, 1334)	1.0
  (9, 1381)	1.0


/usr/lib/python3/dist-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


# Something New, Something Interesting

Let's get funky.

For this, we need

- [ ] Frequent Patterns with split values
    - [x] New Data transformator JSON -> GRAPH
    - [x] Transform to GRAPH
    - [x] Mining frequent patterns ('Initial Rooted Frequent Subtree Mining (without embedding computation) -- With Split Values in Labels.ipynb')
    - [x] New Data transformator CSTRING -> JSON (cString2json.py updated)
    - [x] Transform to JSON ('Find All Occurrences of All Frequent Patterns of Size up to 6.ipynb')
    - [ ] fix missing member problem in SubtreeFeatures

In [6]:
f = open('../forests/rootedFrequentTrees/adult/WithLeafEdgesWithSplitValues/leq6/RF_15_t2.json')
frequentpatterns = json.load(f)
f.close()

In [7]:
%time dsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(map(lambda x: x['pattern'], frequentpatterns[-100:]))

CPU times: user 45.1 ms, sys: 0 ns, total: 45.1 ms
Wall time: 44.6 ms


In [8]:
%time fts = dsf.fit_transform(X)


CPU times: user 9.42 s, sys: 1.91 ms, total: 9.42 s
Wall time: 9.42 s


In [14]:
from sklearn.preprocessing import OneHotEncoder
%time fts_onehot = OneHotEncoder(n_values=dsf.get_n_values()).fit_transform(fts)

CPU times: user 115 ms, sys: 0 ns, total: 115 ms
Wall time: 113 ms


/usr/lib/python3/dist-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


In [16]:
print(X.shape)
print(fts_onehot.shape)
print(fts.shape)
print(Y.shape)

(32561, 64)
(32561, 590)
(32561, 100)
(32561,)


# Classification Performance of Decision Tree Snippet Features vs. Normal Features 

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator

### Linear Regression

In [18]:
model = LinearRegression()
%time fts_onehot_cv_score = cross_val_score(model, fts_onehot, Y, cv=5, scoring='neg_mean_squared_error')
print(fts_onehot_cv_score)

CPU times: user 28 s, sys: 17.7 s, total: 45.8 s
Wall time: 11.7 s
[-0.1138037  -0.1156247  -0.11062547 -0.11274397 -0.11262436]


In [19]:
model = LinearRegression()
%time normalfeatures_cv_score = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')
print(normalfeatures_cv_score)

CPU times: user 1.9 s, sys: 392 ms, total: 2.29 s
Wall time: 644 ms
[-0.11673837 -0.11713078 -0.11536309 -0.11525609 -0.11559877]


### Naive Bayes

In [23]:
model = GaussianNB()
%time fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot.toarray(), Y, cv=5, scoring='f1')
print(fts_onehot_nb_cv_score)

CPU times: user 1.16 s, sys: 276 ms, total: 1.43 s
Wall time: 1.43 s
[0.58989373 0.59108781 0.59519168 0.59317585 0.59445407]


In [24]:
model = GaussianNB()
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 186 ms, sys: 762 µs, total: 187 ms
Wall time: 185 ms
[0.4222318  0.41802575 0.42875817 0.40732665 0.43421624]


### Thresholded Linear Regression

In [38]:
class LinRegClassifier(LinearRegression):
    def __init__(self, threshold=0.5):
        super(LinRegClassifier, self).__init__()
        self.threshold = threshold
    
    def predict(self, X):
        p = super(LinRegClassifier, self).predict(X)
        return (p > self.threshold)

check_estimator(LinRegClassifier)

/usr/lib/python3/dist-packages/sklearn/utils/estimator_checks.py:308: SkipTestWarning: pandas is not installed: not testing for input of type pandas.Series to class weight.
  warnings.warn(str(exception), SkipTestWarning)


AssertionError: 0.3294345956863125 not greater than 0.5

In [26]:
model = LinRegClassifier()
%time fts_onehot_nb_cv_score = cross_val_score(model, fts_onehot, Y, cv=5, scoring='f1')
print(fts_onehot_nb_cv_score)

CPU times: user 27.7 s, sys: 17.1 s, total: 44.8 s
Wall time: 11.3 s
[0.62030905 0.62056213 0.63053097 0.61715559 0.62558223]


In [27]:
model = LinRegClassifier()
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 2.01 s, sys: 453 ms, total: 2.47 s
Wall time: 682 ms
[0.58562691 0.59600614 0.60770111 0.59946133 0.61109044]


## Comparison to DT and RF on Train

In [28]:
model = DecisionTreeClassifier(max_depth=15)
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 891 ms, sys: 2.08 ms, total: 893 ms
Wall time: 892 ms
[0.64163594 0.64901106 0.66395494 0.68107749 0.67190056]


In [31]:
model = RandomForestClassifier(max_depth=15, n_estimators=100)
%time normalfeatures_nb_cv_score = cross_val_score(model, X, Y, cv=5, scoring='f1')
print(normalfeatures_nb_cv_score)

CPU times: user 9.5 s, sys: 12.5 ms, total: 9.51 s
Wall time: 9.52 s
[0.6586169  0.66150442 0.67710229 0.67557535 0.6768559 ]


## Comparison of LinearRegression on DT features -> Should be more or less identical

In [32]:
# load one of the readily available decision trees
f = open('../forests/adult/text/DT_5.json')
dt = json.load(f)
f.close()
dt_fsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)
dt_fts = dt_fsf.fit_transform(X)
dt_fts_onehot = OneHotEncoder(n_values=dt_fsf.get_n_values()).fit_transform(dt_fts)
print(dt_fts_onehot.shape)

(32561, 55)


/usr/lib/python3/dist-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


In [36]:
from sklearn.base import BaseEstimator, ClassifierMixin
class UntrainableDTClassifier(BaseEstimator):
    def __init__(self, dt):
        super(UntrainableDTClassifier, self).__init__()
        self.decisionTreeModel = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)

        tree = self.decisionTreeModel.patterns[0]
        self.linreg_weights = np.zeros(self.decisionTreeModel.get_n_values())
        for i in range(self.decisionTreeModel.get_n_values()):
            try:
                self.linreg_weights[i] = tree.nodes[i].prediction[0]
            except TypeError:
                self.linreg_weights[i] = 0
        
    def fit(self, X, y):
        pass

    def predict(self, X):
        return np.dot(X, self.linreg_weights)

check_estimator(UntrainableDTClassifier)
    
    
model = UntrainableDTClassifier(dt)
print(dt_fts_onehot.shape)
%time 
dtfeatures_lr_cv_score = cross_val_score(model, dt_fts_onehot, Y, cv=5, scoring='f1')
print(dtfeatures_lr_cv_score)

(32561, 55)
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.34 µs


TypeError: 'NoneType' object is not iterable

In [ ]:
a = np.array([1,2,3])
b = np.array([[1,2,3], [1,2,3]])
print(a)
print(b)
print(np.dot(b,a))

## Comparison of NaiveBayes on DT features -> Should be more or less identical

In [21]:
# load one of the readily available decision trees
f = open('../forests/adult/text/DT_5.json')
dt = json.load(f)
f.close()
dt_fsf = DecisionSnippetFeatures.FrequentSubtreeFeatures(dt)
dt_fts = dt_fsf.fit_transform(X)
# dt_fts_onehot = OneHotEncoder(n_values=dt_fsf.get_n_values()).fit_transform(fts)
print(dt_fsf.get_n_values())

55


In [22]:
model = GaussianNB()
%time dtfeatures_nb_cv_score = cross_val_score(model, dt_fts, Y, cv=5, scoring='f1')
print(dtfeatures_nb_cv_score)

CPU times: user 38.7 ms, sys: 0 ns, total: 38.7 ms
Wall time: 38 ms
[0.57308176 0.57518322 0.57555049 0.57035553 0.579     ]
